## Question 7: 

a)

In [155]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import numpy as np

In [146]:
def spec_score(y, y_pred):
    if(len(y) != len(y_pred)):
        print( "Error in spec_score")
        exit(-1)
    fp = 0.
    tn = 0.
    for i in range(len(y_pred)):
        if y_pred[i] == 0 and y[i] == 0:
            tn += 1
        elif y_pred[i] == 1 and y[i] == 0:
            fp += 1
    
    return tn / (fp + tn)


In [187]:
import csv

spam = []
with open('spam.csv') as csv_file:
    spam_file = csv.reader(csv_file, delimiter=',')
    linenum = 0;
    for row in spam_file:
        if(linenum != 0):
            spam.append(row)
        linenum += 1


#Transform Data to numeric type
enc = OrdinalEncoder()
enc.fit(spam)
t_spam = enc.transform(spam)

#Scale Data [0, 1]
scaler = MinMaxScaler()
scaler.fit(t_spam)
s_spam = scaler.transform(t_spam)

#Split data and target vector
y = np.array(s_spam.T[len(s_spam[0]) - 1]) #target vector
x = np.delete(s_spam, len(s_spam[0]) - 1, 1) #data


skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(x, y)

train_data = []
test_data = []
for train_i, test_i in skf.split(x, y):
    x_train, x_test = x[train_i], x[test_i]
    y_train, y_test = y[train_i], y[test_i]
    train_data.append((x_train, y_train))
    test_data.append((x_test, y_test))

b)

In [188]:

model_array = [ KNeighborsClassifier(n_neighbors = 3), 
                KNeighborsClassifier(n_neighbors = 7),
                KNeighborsClassifier(n_neighbors = 11),
                KNeighborsClassifier(n_neighbors = 15),
                DecisionTreeClassifier(max_depth = 5),
                DecisionTreeClassifier(criterion = "entropy", max_depth = 100, min_samples_split = 25),
                GaussianNB()]

for model in model_array:
    print(model)
    split_accuracy = []
    split_auc = []
    split_recall = []
    split_spec = []
    for i in range(len(train_data)):
        model.fit(train_data[i][0], train_data[i][1])
        y_pred = model.predict(test_data[i][0])
        
        split_accuracy.append(metrics.accuracy_score(test_data[i][1], y_pred))
        split_auc.append(metrics.roc_auc_score(test_data[i][1], y_pred))
        split_recall.append(metrics.recall_score(test_data[i][1], y_pred))
        split_spec.append(spec_score(test_data[i][1], y_pred))
    
    
    for i in range(10):
        print("Split: ", i, " Accuracy=", round(split_accuracy[i], 5),
              "AUC=", round(split_auc[i], 5), " Recall=", round(split_recall[i], 5), " Specificity=", round(split_spec[i], 5))
    print("Average Accuracy over 10 splits:", np.mean(split_accuracy))
    print("Average AUC over 10 splits:", np.mean(split_auc))
    print("Average Recall over 10 splits:", np.mean(split_recall))
    print("Average Specificity over 10 splits:", np.mean(split_spec))

KNeighborsClassifier(n_neighbors=3)
Split:  0  Accuracy= 0.76606 AUC= 0.80833  Recall= 0.92958  Specificity= 0.68707
Split:  1  Accuracy= 0.97235 AUC= 0.97222  Recall= 0.97183  Specificity= 0.9726
Split:  2  Accuracy= 0.97696 AUC= 0.97926  Recall= 0.98592  Specificity= 0.9726
Split:  3  Accuracy= 0.96313 AUC= 0.95813  Recall= 0.94366  Specificity= 0.9726
Split:  4  Accuracy= 0.94009 AUC= 0.93377  Recall= 0.91549  Specificity= 0.95205
Split:  5  Accuracy= 0.92627 AUC= 0.91626  Recall= 0.88732  Specificity= 0.94521
Split:  6  Accuracy= 0.92627 AUC= 0.91626  Recall= 0.88732  Specificity= 0.94521
Split:  7  Accuracy= 0.95853 AUC= 0.95833  Recall= 0.95775  Specificity= 0.9589
Split:  8  Accuracy= 0.97235 AUC= 0.96498  Recall= 0.94366  Specificity= 0.9863
Split:  9  Accuracy= 0.85714 AUC= 0.79254  Recall= 0.60563  Specificity= 0.97945
Average Accuracy over 10 splits: 0.9259142603475248
Average AUC over 10 splits: 0.9200087675432898
Average Recall over 10 splits: 0.9028169014084508
Average Sp